In [ ]:
import os
changed_dir = False
if not changed_dir:
    os.chdir("..")
    changed_dir=True

In [20]:
import os
import pandas as pd
from datetime import datetime


def calculate_uwb_metrics(df):
    anchors = df["anchor"].unique()
    mean_dt = df["timestamp"].diff().mean().to_pytimedelta().microseconds / 10**6
    jitter = df["timestamp"].diff().dropna().map(lambda x: x.to_pytimedelta().microseconds / 10**6).diff().dropna().mean()
    # f"UWB freq {1/mean_dt:.2f}Hz | UWB jitter {jitter:.2}s | Positioning freq {1/(mean_dt * len(anchors)):.2f}Hz"
    f_uwb = 1/mean_dt
    f_pos = f_uwb / len(anchors)
    return f_uwb, jitter, f_pos

def calculate_time_difference(start_timestamp, end_timestamp):
    # start_datetime = datetime.fromisoformat(start_timestamp)
    # end_datetime = datetime.fromisoformat(end_timestamp)
    # time_difference = end_datetime - start_datetime
    time_difference = end_timestamp - start_timestamp
    return time_difference.total_seconds()

def process_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    try:
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        if 'timestamp' in df.columns:
            timestamps = df['timestamp'].tolist()
            if len(timestamps) > 1:
                time_difference = calculate_time_difference(timestamps[0], timestamps[-1])
                metrics = calculate_uwb_metrics(df)
                return time_difference, metrics
            else:
                return 0, None
        return 0, None
    except:
        return 0, None
    
def process_directory(directory):
    total_time = 0
    data = {"f_uwb": [], "jitter": [], "f_pos": []}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.csv') or file.lower().endswith('.csv_bad'):
                csv_file_path = os.path.join(root, file)
                time_difference, metrics = process_csv_file(csv_file_path)
                if metrics is not None:
                    f_uwb, jitter, f_pos = metrics
                    data["f_pos"].append(f_pos)
                    data["f_uwb"].append(f_uwb)
                    data["jitter"].append(jitter)

                total_time += time_difference
    metrics_df = pd.DataFrame(data)
    return total_time, metrics_df.mean()


In [21]:
directory_path = "/home/piotr/Documents/mgr/uwb-rs/data-processing"
total_time, metrics = process_directory(directory_path)
print(f"Total time in seconds: {total_time}")
print(f"Total time in minutes: {total_time / 60}")
print(f"Total time in hours: {total_time / 60 / 60}")
metrics

/tmp/ipykernel_87600/960760985.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["timestamp"])


Total time in seconds: 49742.332465
Total time in minutes: 829.0388744166667
Total time in hours: 13.81731457361111


f_uwb     55.335672
jitter     0.000072
f_pos     27.738318
dtype: float64